In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import torch 


In [7]:
df = pd.read_csv('../data/data.csv')

df.head()

,text
0,Cricket is a bat-and-ball game played between ...
1,The batting side scores runs by striking the b...
2,"Means of dismissal include being bowled, when ..."
3,They communicate with two off-field scorers wh...
4,"In addition to the basic kit, some players wea..."


In [11]:
texts = df['text']

In [14]:
vectorizer = CountVectorizer(ngram_range=(1, 3))

ngram_features = vectorizer.fit_transform(texts)

# Calculate the n-gram scores as the sum of n-gram features for each text
ngram_scores = ngram_features.sum(axis=1)

# Calculate the threshold as the maximum n-gram score
threshold = max(ngram_scores)

In [15]:
threshold

matrix([[564]])

In [9]:
vectorizer = CountVectorizer(ngram_range=(1, 3))

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def calculate_ngram_score(chunk):
    ngrams = vectorizer.transform([chunk]).toarray()
    ngram_score = np.sum(ngrams)
    return ngram_score

def calculate_weak_llm_score(chunk):
    tokens = tokenizer(chunk, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokens)
        embeddings = output.last_hidden_state.mean(dim=1).squeeze(0).numpy()
    weak_llm_score = np.mean(embeddings)
    return weak_llm_score

ngram_scores = [calculate_ngram_score(chunk) for chunk in ans]
weak_llm_scores = [calculate_weak_llm_score(chunk) for chunk in ans]

ngram_threshold = np.percentile(ngram_scores, 90)
weak_llm_threshold = np.percentile(weak_llm_scores, 90)

threshold = (ngram_threshold + weak_llm_threshold) / 2

print(f'N-gram Threshold: {ngram_threshold}')
print(f'Weak LLM Threshold: {weak_llm_threshold}')
print(f'Combined Threshold: {threshold}')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 │   weak_llm_score = np.mean(embeddings)                                                    │
│   17 │   return weak_llm_score                                                                   │
│   18                                                                                             │
│ ❱ 19 ngram_scores = [calculate_ngram_score(chunk) for chunk in ans]                              │
│   20 weak_llm_scores = [calculate_weak_llm_score(chunk) for chunk in ans]                        │
│   21                                                                                             │
│   22 ngram_threshold = np.percentile(ngram_scores, 90)                                           │
│                                                                                                  │
│ in <listcomp>:19                                                                                 │
│                                                                                                  │
│   16 │   weak_llm_score = np.mean(embeddings)                                                    │
│   17 │   return weak_llm_score                                                                   │
│   18                                                                                             │
│ ❱ 19 ngram_scores = [calculate_ngram_score(chunk) for chunk in ans]                              │
│   20 weak_llm_scores = [calculate_weak_llm_score(chunk) for chunk in ans]                        │
│   21                                                                                             │
│   22 ngram_threshold = np.percentile(ngram_scores, 90)                                           │
│                                                                                                  │
│ in calculate_ngram_score:7                                                                       │
│                                                                                                  │
│    4 model = DistilBertModel.from_pretrained('distilbert-base-uncased')                          │
│    5                                                                                             │
│    6 def calculate_ngram_score(chunk):                                                           │
│ ❱  7 │   ngrams = vectorizer.transform([chunk]).toarray()                                        │
│    8 │   ngram_score = np.sum(ngrams)                                                            │
│    9 │   return ngram_score                                                                      │
│   10                                                                                             │
│                                                                                                  │
│ /home/arrry/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:142 │
│ 5 in transform                                                                                   │
│                                                                                                  │
│   1422 │   │   │   raise ValueError(                                                             │
│   1423 │   │   │   │   "Iterable over raw text documents expected, string object received."      │
│   1424 │   │   │   )                                                                             │
│ ❱ 1425 │   │   self._check_vocabulary()                                                          │
│   1426 │   │                                                                                     │
│   1427 │   │   # use the same matrix-building strategy as f

In [10]:


X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

vectorizer = CountVectorizer(ngram_range=(1, 2))  # Adjust the n-gram range
X_train_ngrams = vectorizer.fit_transform(X_train)
X_test_ngrams = vectorizer.transform(X_test)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def extract_features_from_text(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokens)
        embeddings = output.last_hidden_state.mean(dim=1).squeeze(0).numpy()
    return embeddings

X_train_distilbert = np.array([extract_features_from_text(text) for text in X_train])
X_test_distilbert = np.array([extract_features_from_text(text) for text in X_test])

X_train_features = np.hstack((X_train_ngrams.toarray(), X_train_distilbert))
X_test_features = np.hstack((X_test_ngrams.toarray(), X_test_distilbert))

threshold = 0.5

lr_model = LogisticRegression()
lr_model.fit(X_train_features, y_train)

accuracy = lr_model.score(X_test_features, y_test)
print(f'Accuracy: {accuracy}')

new_text = "This is a test sentence."
new_text_ngrams = vectorizer.transform([new_text]).toarray()
new_text_distilbert = extract_features_from_text(new_text).reshape(1, -1)
new_text_features = np.hstack((new_text_ngrams, new_text_distilbert))
prediction = lr_model.predict(new_text_features)[0]

if prediction == 1:
    print("The text is likely LLM-generated.")
else:
    print("The text is likely not LLM-generated.")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/arrry/anaconda3/envs/ML/lib/python3.9/site-packages/pandas/core/indexes/base.py:3653 in    │
│ get_loc                                                                                          │
│                                                                                                  │
│   3650 │   │   """                                                                               │
│   3651 │   │   casted_key = self._maybe_cast_indexer(key)                                        │
│   3652 │   │   try:                                                                              │
│ ❱ 3653 │   │   │   return self._engine.get_loc(casted_key)                                       │
│   3654 │   │   except KeyError as err:                                                           │
│   3655 │   │   │   raise KeyError(key) from err                                                  │
│   3656 │   │   except TypeError:                                                                 │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:147                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:176                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7080                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7088                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'label'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0    │
│    2                                                                                             │
│    3 vectorizer = CountVectorizer(ngram_range=(1, 2))  # Adjust the n-gram range                 │
│    4 X_train_ngrams = vectorizer.fit_transform(X_train)                                          │
│                                                                                                  │
│ /home/arrry/anaconda3/envs/ML/lib/python3.9/site-packages/pandas/core/frame.py:3761 in           │
│ __getitem__                                                                                      │
│                                                                                                  │
│    3758 │   │   if is_single_key:                                                                │
│    3759 │   │   │   if self.columns.nlevels > 1:                                                 │
│    3760 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  3761 │   │   │   indexer = self.columns.get_loc(key)                                          │
│    3762 │   │   │   if is_integer(indexer):                                                      │
│    3763 │   │   │   │   indexer = [indexer]                                                      │
│    3764 │   │   else:                                                                            │
│                                                                        